![CH6-ADS.png](.\Media\CH6-ADS.png)

# <span style="color:#cc5500;">Capacity Planning</span>

## <span style="color:#cc5500;">Introduction</span>

Planning for the future growth of your applications and databases should start early and then be addressed on a frequent basis.  In fact, capacity planning is all about the growth of your systems.  Capacity planning of a system can be thought of in terms with some of the following questions:

- What is the number of users in the system? 
- Can the server scale-up, or will it have to be scaled-out?
- What is the total number of databases this server will host?
- How many Instances will this server host?
- Most SQL Workloads on-premises can the server be virtualized, but is this new workload I am considering, so large they will have to run on bare metal?
- Do I have sufficient disk space, and will the disk array perform to my requirements under load?

This notebook will provide guidance on how to plan for capacity and provide formula’s where possible and ‘Rules of Thumb’ as a best practice to help you make good decisions related to capacity planning.

## <span style="color:#cc5500;">Capacity Planning for Table Growth</span>

Capacity planning for table and database growth can at times be a daunting task.  However, armed with knowledge about SQL Server data types and storage structures, and armed with an understanding of your applications, it is possible to estimate growth rates.  Use the following ‘Fictional’ example to provide a framework on how to capacity plan table growth.

Example: Assume that you have a database table named “Execution Log”.  Assume this table logs a record every time a user runs a report.  This table will store metadata about the report that was run in several table columns.  As the volumes of reporting increases the amount of logged data will increase.

### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Click the run icon below
2. If ADS prompts you for a connection, enter the correct SQL Server and authentication account
3. View the results of the query by scrolling down to the results set

Create the test database



In [ ]:
USE [master];
GO
--If exists, drop the Company database
IF DATABASEPROPERTYEX (N'Company', N'Version') > 0
BEGIN
	ALTER DATABASE [Company] SET SINGLE_USER
		WITH ROLLBACK IMMEDIATE;
	DROP DATABASE [Company];
END
GO
--Modify the file path for your environment and create the database
CREATE DATABASE [Company] ON PRIMARY (
    NAME = N'Company',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company.mdf')
LOG ON (
    NAME = N'Company_log',
    FILENAME = N'C:\Program Files\Microsoft SQL Server\MSSQL14.VIPER\MSSQL\DATA\Company_log.ldf');
GO

In [ ]:
-- Lets create a Table 
USE [Company];
GO
CREATE Table [Execution Log](
	[InstanceName] NVARCHAR (38),
	[ReportID] UNIQUEIDENTIFIER,
	[UserName] NVARCHAR (260),
	[RequestType] BIT,
	[Format] NVARCHAR (26),
	[Parameters]	NTEXT,
	[TimeStart] DATETIME,
	[TimeEnd] DATETIME,
	[TimeDataRetrieval] INT,
	[TimeProcessing] INT,
	[TimeRendering] INT,
	[Source] TINYINT,
	[Status] NVARCHAR (32),
	[ByteCount] BIGINT,	
	[RowCount] BIGINT);	

The following query exposes the needed information about the table data types, and for this example the “Execution Log” table in the fictional database to help determine how big the table will get based on a ‘given’ transaction rate.

In [ ]:
SELECT	Column_ID, [Name], Type_name(User_Type_ID)
	AS User_Type_Name, Max_Length
FROM sys.columns
WHERE object_id=object_id('Execution Log')

Your results should look like this:

![CH6-ExecutionLog.jpg](.\Media\CH6-ExecutionLog.jpg)

One would think that a simple sum of the Max\_Length\_bytes column shows that each record in the table would take up 790 bytes of storage space.  This however, IS NOT the true size. Further explanation is necessary. The “Execution Log” example table contains both fixed length data types and variable length data types. The fixed length columns are listed and a simple sum of the length produces an accurate measure of 62 bytes.

Fixed Length Columns

![CH6-FixedLength.jpg](.\Media\CH6-FixedLength.jpg)

The Variable length data types however, are a bit more complex and require the DBA to do some educated estimation.

![CH6-VariableLength.jpg](.\Media\CH6-VariableLength.jpg)

### <span style="color:#cc5500;">Some Rules to Consider</span>

Rule 1:  Variable data types like the Unicode nvarchar are 2(X) the number of characters entered 

Rule 2:  For nvarchar data types add 2 bytes of row overhead

Rule 3:  Data types of ntext are 2(X) the number of characters entered

Rule 4:  If any of the column data types in the table make use of variable data types, then add 2 more additional bytes to the total for row overhead.

- There are 4 columns of type nvarchar in this table totaling 712 bytes
- Since these are variable length types, it is not likely that all records will take up the maximum possible data length, so we will use a “guestimate” that an average of 80 % of the 712 bytes will be used thus generating an estimate of 570 bytes for the four nvarchar columns.
- There are 8 bytes for the row overhead associated with the four nvarchar columns
- We will use an “guestimate” average of 100 characters of ntext times 2 generating 200 bytes

To determine how much space a single record will take in the “Execution Log” table is

(62) + (570) + (8) + (200) + if any columns have variable length types add (2) more bytes because if there are any variable length columns in a table, \[Rule 4\] two more bytes are required =

842 bytes per record

8096 bytes can be stored in a single 8k data page.  8096 / 842 bytes = 9 rows of “Execution Log” data can fit in a single 8K data page

### <span style="color:#cc5500;">Putting it all together</span>

Assume your company runs an average of 100,000 user reports a day

100,000 X 842 = 84,200,000 bytes

84,200,000 bytes = 82.2 MB

82.2 MB X 60 days = 4,942 MB = 4.82 GB (rounded)

BUT as we learned above, we can only fit 9 records per data page.  100,000 / 9 = 11,111 data pages (rounded)

11,111 X 8192 = 91,021,312 Bytes

91,021,312 = 86.8 MB

86.8 MB X 60 days = 5,208 MB = 5.08 GB (rounded)

With this information, it can be estimated in this example, that if your company executes 100,000 user reports per day, the fictional “Execution Log” table will grow by 5.08 GB in a 60 day period.

## <span style="color:#cc5500;">Capacity Planning for System Databases</span>

SQL Server system databases (master, model, msdb, tempdb) by default are fairly small.  Generally they are less than 30MB.  Even though limited changes take place on system databases (tempdb being the exception) master, model, and msdb should be backed up on a periodic basis or when any change occurs.  Tempdb is recreated every time SQL Server is restarted and so there is not a need for it to be backed.  SQL Server prevents backup and restore operations of tempdb.  It is recommended to place all system databases on a dedicated LUN.  The I/O work load on tempdb can become very high depending on the application, number of databases, and SQL statements executed by the instance.

- master database:  no modification or optimization is necessary other than the recommendation to place it on a fault tolerant disk
- msdb database:  no modification or optimization is necessary other than the recommendation to place it on a fault tolerant disk
- model:  This can be modified by a DBA for administration purposes to change the default collation, size, growth rate, etc.
- tempdb:  This should be optimized.  See the companion notebook on how and why tempdb should be optimized

## <span style="color:#cc5500;">Capacity Planning for User Databases</span>

Gaining an understanding of the space and throughput requirements for user-defined databases will depend upon the measurements that you collect at periods of peak load on the production system. 

Data Files

- The total physical size of database files, are usually measured in GB.  It is a common practice for large tables to be physically separated from each other using file groups and secondary data files placed on separate LUN’s.
- It is recommended that the growth rate of database files be set to an MB value rather than a percentage value.  If a database is only 500MB and it is set to grow by 10%, it is not a major concern.  However, if the database is 40GB and it is set to grow by 10%, then this situation could present a very big capacity planning problem.  Setting the growth rate to MB provides predictability.
- Use careful planning when making use of database snapshots as the snapshot will grow when updates are made to the database that is snapped.

Transaction Log Files

- Will generally stabilize at a predictable size depending on configured recovery mode and how often transaction log is backed up.
- Will generally only grow in the beginning until a balance is reached between the arrival rate of new transactions and frequency log is backed up.
- Each time transaction log is backed up the log file will be logically truncated and made available for new transactions.  Physical file size will not change.
- Administrative tasks and data modification tasks can cause the log to grow so plan for extra space for highly transactional databases

How to collect sizing information

- Check database size using the sp\_spaceused system stored procedure
- Use Performance Monitor or Dynamic Management Views to monitor size of a database’s file

## <span style="color:#cc5500;"><br></span>

## <span style="color:#cc5500;">Capacity Planning for Indexes</span>

The physical data in a table without a clustered index is stored in a ‘heap’, and the data in a table with a defined clustered index is stored with in the index.  Most DBA's have developed periodic index maintenance schedules that rebuild or reorganize indexes based upon a fragmentation threshold.   The ability to create and rebuild an index is one of the most frequent database management tasks.  Some of the key factors to consider while you devise an index maintenance plan are to think of concurrency, performance, and required resources.

The benefits of defragmenting indexes are without question.  However it is important to know that index operations will consume space and make use of server resources.   

Most DBA's makes use of ONLINE index rebuilding and reorganizing strategies.  Using the ONLINE option will provide maximum concurrency, but it uses more resources and takes longer to complete than an OFFLINE option (which reduces concurrency).    

Some of the factors that will influence how much space might be required for index operations are:

- If the index is a narrow single column index or a wide index that encompasses many of the columns in the table
- If there are transactions on the table that are concurrent with the online index create or rebuild, row versions are maintained in the version store.  Row versioning is used for online index create and rebuild operations, and tempdb will need additional space on the disk drive where tempdb is located for the version store. 
- When you rebuild a clustered index, no sorting is required because the data is already sorted. Therefore, no temporary space is required for sort runs. 
- However, when you create a clustered index, sorting is required, and you will use temporary space on the disk drive that the database is located.

After tempdb has been optimized and moved to a dedicated fault tolerant disk array, consider taking advantage of a feature of SQL Server to help control how SQL Server manages the temporary space during an index operation.  SQL Server provides SORT\_IN\_TEMPDB option.  SQL Server uses temporary storage for sorting and other intermediate tasks while creating or rebuilding an index.  When SORT\_IN\_TEMP is set to ON, the temporary storage is then pushed to the TEMPDB database.  

You can monitor the resources that tempdb will use during an SORT\_IN\_TEMPDB index operation by using the following query as an example:

In [ ]:
USE [SalesDB]
SELECT * FROM sys.dm_db_session_space_usage WHERE SESSION_ID = @@spid
GO
ALTER INDEX [SalesPK] ON [dbo].[Sales]
REBUILD PARTITION = ALL WITH (SORT_IN_TEMPDB = ON)
GO
SELECT * FROM sys.dm_db_session_space_usage WHERE SESSION_ID = @@spid


Testing has shown that the temporary storage of some types of indexing operations have consumed more than 2x the size of the database to complete.  

It is recommended that you perform testing on different types of indexing to understand the concurrency, performance, and required resources.

A strategy to control ‘where’ the temporary resources are needed during indexing operations would be to consider separating large tables from each other into unique LUN’s.  This can be accomplished by dropping a table index and re-creating it on a separate LUN.

## <span style="color:#cc5500;">Capacity Planning Memory for SQL Server</span>

The Microsoft website has a good amount of information on how SQL Server memory works and some general recommendations for the configuration of memory on a database server.  Setting SQL Max server memory is different for every single server and every application work load as it is based upon the unique characteristics for each environment.  These include:

- The number of CPU’s
- Whether it is a 32 bit (x86), a 64 bit (x64) or an Itanium (ia64) operating system
- The amount non-SQL applications that are installed
- The memory requirements for these non-SQL applications.

### <span style="color:#cc5500;">The Formula</span>

A very frequent question that I receive from customers is how to apply some sort of formula to the Max Server Memory setting instead of just making a best guess or pulling opinions from a popular website.  I developed this formula while working closely with the Microsoft Product Group, Microsoft Premier Support, the SQL and SharePoint CAT teams, as well as drawing from years of customer facing experience.  Use this formula as guidance on how much memory to dedicate to your SQL Server instance.  

Set ‘Max Server Memory’ formula

SQL Max Memory = 

> TotalPhysMem
> 
> <span style="background-color: rgb(33, 33, 33);">- (NumOfSQLThreads x ThreadStackSize)</span>
> 
> <span style="background-color: rgb(33, 33, 33);">- (1GB x CEILING(NumOfCores/4))</span>

Value Examples  

> NumOfSQLThreads \= 
> 
> <span style="background-color: rgb(33, 33, 33);">256 + (NumOfProcessors** - 4) x 8&nbsp;&nbsp;</span> 
> 
> <span style="background-color: rgb(33, 33, 33);">[note: 256 listed above is for a 4CPU 32bit server ]</span>

ThreadStackSize   

> \=1 MB on x86

> \= 2 MB on 64-bit (x64)

> \= 4 MB on 64-bit (IA64)

> \*\*If NumOfProcessors \> 4, else 0

To solve the first piece of this formula, it is important to know the start-up worker threads for the particular server to be configured.  This is the information that we publish on our website regarding the default number of worker threads created at start up.

  ![CH6-CPUNum.jpg](.\Media\CH6-CPUNum.jpg)

  

However, it can be difficult to find information for larger systems.  The numbers listed in the table above are for small to midsized servers ranging from less than 4 and up to 32 CPUs.  The example below is how to get the start-up worker threads for a 64 CPU x64 system we use:

> 512 + ((No of CPU’s – 4) \* 16)

The DBA can also run this query to see the current number for your server:

In [ ]:
select max_workers_count from sys.dm_os_sys_info

### <span style="color:#cc5500;">Practical Example</span>

This is an example for the purpose of providing clarity to the above formula.   Assume your customer has a large (x64) bit production server with 512GB of RAM and 64 CPU’s

Applying the above Formula:

<span style="color: rgb(238, 236, 225);">SQL Max Memory =</span>

 <span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 524288 MB</span>

 <span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - (1472 X 2MB)&nbsp;&nbsp; [<i>default no. of worker threads is</i> 1472 X 2 MB <i>for x64 bit</i>]</span>

 <span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - 1024MB X 64/4) [1GB times the ceiling of the equation (CPU’s ÷ 4)]</span>

<span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2"><br></span>

 <span style="color: rgb(238, 236, 225);">&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 524,288 MB</span>

 <span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; -2,944 MB</span>

 <span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <u>-16,384 MB</u></span>

 <span style="mso-fareast-font-family:
&quot;Times New Roman&quot;;mso-bidi-font-family:&quot;Segoe UI&quot;;color:#EEECE1;mso-themecolor:
background2">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <b>504,960 MB</b></span>

The end result appears to be, that the DBA would set Max Server Memory to <span style="color:#cc5500;">504,960 MB or 493.1 Gigabytes</span>.  This would be <span style="color:#cc5500;">incorrect.</span>  The Max Server Memory setting is for the SQL Server Buffer Pool only.  There are other processes within SQL Server that are outside of the Buffer Pool that require memory.  So, we have to subtract the overhead of memory required for other SQL processes and non-SQL applications that may be running on this server.  For example, the Lazy Writer, SQL backups, antivirus software, perfmon monitoring, host intrusion detection software (HIDS), or profiler tracing tools provided by Microsoft, Altiris agents, and so on.  Remember, these are all either internal SQL processes, or non-SQL applications and these other non-SQL resources need to be accounted for as they also require the use of some of the total server memory to run in a healthy way.  After testing, you may discover that these other non-SQL applications need a ‘hypothetical’ 1450 MB of RAM (different for your actual server), so you would then, subtract that amount from the MAX Server memory setting.

- Determine the amount of memory headroom that your current server has and how much memory the non-SQL applications require.
    - Use Perfmon Memory: Available Mbytes for at least one week.
    - SQL Server Memory Manager: Memory Grants Pending (should be 0)
    - Use DBCC Memory DBCC memorystatus
    - Helpful link here: [Use DBCC MEMORYSTATUS to monitor memory usage - SQL Server | Microsoft Docs](https://docs.microsoft.com/en-US/troubleshoot/sql/performance/dbcc-memorystatus-monitor-memory-usage)
    - Use Task Manager
- Use the formula provided above to calculate max server memory for Production SQL Servers
- Use Formula and the other values listed above to help you configure memory for all of your other SQL Servers (both 32 bit and 64 bit)
- Review the documentation that I will provide on the performance counters to capture, the thresholds, and the significance of each counter value.
- Use the following query to assist

In [ ]:
SELECT
(physical_memory_in_use_kb/1024) AS MemoryUsed_MB,
(locked_page_allocations_kb/1024) AS LockedPagesUsed_MB,
(total_virtual_address_space_kb/1024) AS TotalVAS_MB,
process_physical_memory_low,
process_virtual_memory_low
FROM sys.dm_os_process_memory;

The process\_physical\_memory\_low column indicates that the process is responding to a low physical memory notification.  The process\_virtual\_memory\_low column indicates that a low memory condition has been detected.

### <span style="color:#cc5500;">Final Example from a Real-World Situation</span>

Even with powerful (x64) bit systems provisioned with large amounts of RAM, and with the LPIM “Lock Pages in Memory” privilege set, the Windows Operating System will still make SQL Server dump its memory if it must.  This is almost always due to improper configuring of memory for the SQL Server Buffer Pool.  Such an event would cause a significant impact to performance and end user experience.

Not long ago, a DBA for one of our very large banking customers had a brand new (X64) bit server with 64GB RAM and the LPIM privilege in place.  This was supporting a critical production application.  The DBA did not account for the non SQL applications and left the MAX server memory to the ‘default’ setting as was his practice with all of his SQL Servers.  He connected to the server via Terminal Services, launched two instances of Perfmon to do some health checks and opened up a SQL Server Management Studio query.  He queried a SQL DMV using the CROSS APPLY function.  As soon as the simple query was run, the memory available for the operating system reached a tipping point and the system performed a memory stack-dump and the database server was unresponsive for about a 9 minute period until the server could recover from the incident.  The banks customers were unable to access their banking accounts during this 9 minute period.  The situation would have been avoided had the max server memory been set correctly and this proved to be an incredibly painful learning experience for the DBA.

Once you are ready make adjustments to the configuration of its servers, use the above formula as guidance in conjunction with thorough testing and analysis to configure the optimal setting.  It is important to make sure that the non-SQL applications have sufficient memory headroom.

  

## <span style="color:#cc5500;">Capacity Planning Memory for Storage</span>

The objective is to aim for \< 10 millisecond response time for disk reads and writes.

- Most Storage Area Networks support the use of zoning which allows the SAN administrator to dedicate entire disks in the SAN to the LUN for the purpose of isolating the IO on a drive to SQL Server. 
- You should select the largest appropriate LUN possible as this will distribute the IO better over the multiple disks which comprise your LUN.
- Try to ensure that your log files are on a separate LUN consisting of a dedicated disk.  Log files are written in sequential patterns, unlike data files. Have a separate LUN for database backups.  Do not place data files or log files on the same LUN as backup files as a backup operation can completely ‘soak’ the I/O subsystem and this could source of great contention.
- Understand the number of physical disks backing each LUN and which LUNs share the same physical spindles.    
- Understand any sharing of physical disks between multiple servers and what applications are running on each server.  Problems can arise when different servers share the same physical spindles and have very different IO characteristics (i.e. Exchange and SQL Server) 

Our recommendation is to use RAID 10 both for logs and data/index files.  When cost or space is an issue you could allocate data and index files on RAID 5 or equivalent; however, we strongly recommend that you keep transaction logs on RAID 10.  RAID 10 offers better availability than RAID 5 and better performance especially for write-intensive applications.  Some of our customers have reported up to 50% adverse impact on write intensive work load when moving to RAID 5. The impact on performance of RAID 5 is dependent on the hardware vendor’s implementation of RAID 5 and the characteristics of your particular work load.  You should not assume that you will get a 50% adverse impact – it could be higher or lower.  

There are many reasons to separate data from logs and rooted in the basic design of a database system is that the transaction log fundamentally takes advantage of sequential IO.  The perceived “magic” of a SAN doesn’t change this and one cannot mix random IO in on any drive that services a transaction log and expect the magic of a SAN to completely overcome this design.  If you are rigorous in laying out your LUNs in the SAN, you can guarantee that the heads will never move off for unrelated IO because there will be no unrelated IO in that LUN.  If you set up a stripe set with a set of spindles, and dedicate that set to only one LUN, which is used for the transaction log exclusively, there is nothing to disturb the heads.   

Some of Microsoft’s customers have to make a decision of ‘trade-offs’.  They sometimes have shared storage with other applications or may have little choice but to mix data, logs, or tempdb on the same LUN’s.  

- In a perfect situation transaction logs should always live completely on their own drive volumes.  However, as soon as 3 or 4 user databases exist on a multi-node cluster, you simply may not have enough drive letters available to host each log file and data file on its own drive letter.
- SAN administrators will tend put up very strong resistance to creating many small LUNS to host log files, especially if the SAN in question is using relatively large drives like the 144GB variety that's currently popular. 
- If you truly cannot segregate transaction logs, then at least you can attempt to put yourself in a situation in which no more than 1 log file is placed on each available drive, and also make sure that the log does not share a drive with data files within the same database.  For example, don’t have data files for database "A" placed on the drive on which you place the log file for database "A".